# Exercise Sheet 05: Recommender Systems

**Introduction to Data Mining WS23/24**  
**Bielefeld University**  
**Alina Deriyeva, Benjamin Paaßen**  
**Exercise Sheet Publication Date: 2024-01-08**  
**Exercise Sheet Submission Deadline: 2024-01-19, noon (i.e. 12:00)**, via **moodle** (please do not use e-mail submissions anymore).

**NOTE** The use of language models/AI tools is permitted IF you notify us of the use (just indicate it in the respective task) and are still able to understand and present your results. We also appreciate it if you link to a chatlog of the interaction with the language model/AI tool so that we can understand better how students tend to use these tools.

**PLEASE INDICATE ALL AUTHORS OF THE SUBMISSION IN THIS FIELD**

### Preamble: Data set

Consider the data set in `sheet05_data.csv`. This data set contains the study progress of 50 students in a fictional university bachelor computer science dgree.

Each row corresponds to one attempt of one student at a course. The first column contains the student index $i$, the second column the time step $t$, the third column the course index $j$, and the fourth column the grade the student achieved, where -2 means failed, -1 means passed with a not so good grade, +1 means passed with a good grade, and +2 means passed with an excellent grade.

The courses are the following.

In [ ]:
courses = ['A&D', 'math', 'programming', 'technical CS', 'theoretical CS', 'robotics', 'machine learning', 'data mining', 'software engineering']
grades  = ['failed', 'passed', 'good', 'excellent']

The following codes loads the data and prints the progress for the first student.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.loadtxt('sheet05_data.csv', skiprows = 1, delimiter = '\t', dtype=int)

t = 0
print('progress for student 0')
while True:
    i = X[t, 0]
    if i > 0:
        break
    j = X[t, 2]
    g = X[t, 3]
    print('time %d: \"%s\" with grade \"%s\"' % (t, courses[j], grades[g]))
    t += 1



## Markov chains

In the following tasks, we will apply a Markov chain to this data set. A Markov chain is defined by two types of parameters: For each course $j$, we have a starting probability $\pi_j$. And for each pair for courses $(j, k)$, we have a probability $p_{j, k}$ that indicates the likelihood of attempting course $k$ after course $j$.

### Task 05.01

Compute a Markov Chain based on the given data set. In particular.

1. Generate a vector `pi` with `len(courses)` entries where `pi[j]` is the number of times course `j` was attempted as first course.
2. Divide `pi` by its sum to obtain probabilities.
3. Generate a matrix `P` with `len(courses) x len(courses)` entries where `P[j, k]` is the number of times course `k` was attempted after course `j`.
4. Divide each row of `P` by its sum to obtain probabilities.

### Task 05.02

Try to understand the following code. Then, execute it.

In [ ]:
print("for a student who has not attempted any course yet, the Markov chain would predict next:")
for j in range(len(courses)):
    print("  \"%s\" with probability %d%%" % (courses[j], 100 * pi[j]))

for last_j in range(len(courses)):
    print("for a student who has attempted \"%s\" last, the Markov chain would predict next:" % courses[last_j])
    for j in range(len(courses)):
        print("  \"%s\" with probability %d%%" % (courses[j], 100 * P[last_j, j]))

### Task 05.03

1. Consider a student who has attempted no course, yet. What are the probabilities, according to the Markov chain, to attempt "theoretical CS", "robotics", "machine learning", "data mining", or "software engineering", next? How do you explain these probabilities?
2. Consider a student who has attempted "math" last. What is the probability, as predicted by the Markov chain, to attempt "math" again? How do you explain this probability?

**Answer:**

### Task 05.04

Assume we wanted to use this Markov chain for recommendations by always recommending the course with the highest probability.

Discuss, whether this recommendation scheme makes sense for this data set. In particular:
* Explain at least one advantage of Markov Chain-based recommendation for this data set.
* Explain at least three problems of Markov Chain-based recommendation for this data set.

**ANSWER:**

**Advantage:**

**Disadvantages:**

## Collaborative filtering

User-based collaborative filtering tries to recommend courses to a student based on the grades of students who achieved similar grades previously.

More precisely, let $g_1, \ldots, g_m$ be the grades of a new student for each course, where $g_j = 0$ if a course was not yet attempted. And let $g_{i, 1}, \ldots, g_{i, m}$ be the grades of student $i$ for each course.

Then, user-based collaborative filtering generates recommendations as follows.

1. Compute the similarity $s_i$ of the grades of the new student on any course to the grades of prior students $i$. For example, we can use the cosine similarity:
\begin{equation}
s_i = \frac{\sum_{j=1}^m g_j \cdot g_{i, j}}{\sqrt{\sum_{j=1}^m g_j^2} \cdot \sqrt{\sum_{j=1}^m g_{i, j}^2}}.
\end{equation}
2. We compute the estimated rating of the new student for course $j$ via the equation:
\begin{equation}
g_j \approx \frac{\sum_{i : g_{i, j} \neq 0}^N s_i \cdot g_{i, j}}{\sum_{i : g_{i, j} \neq 0} s_i},
\end{equation}
in other words we take the weighted average grade of all students who have attempted the course.

### Task 05.05

Compute a matrix $G$ with one row per student and one column per course with entries $g_{i, j}$ as defined above, i.e. the grade of student $i$ on course $j$ in their final attempt on the course, where $g_{i, j} = 0$ if student $i$ has not yet attempted course $j$. Print the matrix.

### Task 05.06

Write a python function that takes a vector of grades as input and returns a copy of this vector where each zero entry is replaced with the collaborative filtering estimate of the grade according to the scheme above.

Use your function to print the estimated grades for two fictional students:
1. A student who has achieved an excellent grade in programming and good in technical CS as well as A&D but only passing in math and has not yet attempted any other course.
2. A student who has achieved an excellent grade in math and A&D but only passing in technical CS and programming and has not yet attempted any other course.

Which course would you recommend for these two students, respectively, given the grade estimates?

**ANSWER:**

### Task 05.07

Discuss whether collaborative filtering makes sense for this kind of data. In particular:

* Explain at least one advantage of collaborative filtering for this data set.
* Explain at least two problems of collaborative filtering for this data set.

**ANSWER:**

**Advantage:**

**Disadvantages:**

## Matrix factorization

Given an $N \times m$ grading matrix $G$, matrix factorization attempts to find two matrices, namely a $N \times K$ matrix of student rperesentations $U$ and a $K \times m$ matrix of course representations $V$, such that the Frobenius norm $\lVert G - U \cdot V \rvert_F$ is minimized for a given, small $K$. This is achieved via singular value decomposition.

A singular value decomposition returns three output arguments:
1. The matrix of eigenvectors $\tilde U$ of $G \cdot G^T$.
2. The matrix of eigenvalues $\Lambda$ of $G^T \cdot G$.
3. The matrix of eigenvectors $\tilde V$ of $G^T \cdot G$.

The original matrix $G$ can be losslessly reconstructed via $\tilde U_{:, :m} \cdot \Lambda \cdot \tilde V$, taking only the first $m$ columns of $\tilde U$.

For matrix factorization, we decide on a number of $K \ll m$ latent factors and define the student representations as $U := \tilde U_{:, :K} \cdot \sqrt{\Lambda_{:K, :K}}$ and the course representations as $V := \sqrt{\Lambda_{:K, :K}} \cdot \tilde V$.

### Task 05.08

Provide a python function which takes a $N \times m$ grade matrix $G$ as well as a number of latent components $K$ as input and returns the $N \times K$ matrix of student representations $U$ and the $K \times m$ matrix of course representations $V$.

Use the function `np.linalg.svd` to perform the singular value decomposition.

### Task 05.09

Provide a plot with the number of latent dimensions $K$ on the x axis and the reconstruction error $\lVert G - U \cdot V \rVert_F$ on the y axis for $K$ between 1 and 9.

### Task 05.10

Provide two scatter plots, one for the student representations $U$, and one for the course representations $V$, in $K$ dimensions. The course representations should be labeled with the course names.

### Task 05.11

Make a copy of the original grading matrix $G$.
In the copy, fill in zeros using the entries of the product $U \cdot V$ for $K = 2$.
Plot the resulting grading matrix.

### Task 05.12

Discuss whether matrix factorization makes sense for this kind of data. In particular:

* Explain at least one advantage of matrix factorization for this data set.
* Explain at least two problems of matrix factorization for this data set.

**ANSWER:**

**Advantage:**

**Disadvantages:**